In [1]:
import QuantLib as ql
import numpy as np

# Valuation of CMS coupon - Description

This notebook shows valuation of a single CMS coupon using Linear Terminal Swap Rate model in `QuantLib`, based on gaussian model subject to a mean reversion.


Scenario: today $t=0$ is `2022-10-20`. We value a coupon in a CMS swap. Details:
- the CMS coupon is based on a 5Y-swap rate (swap 6M Euribor vs annual fixed rate) observed at $T=T_0=2Y$, i.e. `2024-10-20` at which the CMS coupon starts to accrue,
- the accrual period ends by $T_p=3Y$, i.e. `2025-10-20`. On the same date the CMS coupon is paid out,
- shifted lognormal model with $60\%$ volatility (shift = $2\%$) is used for the dynamics of forward swap rate in the convexity adjustment computation,
- forecasting curve is flat $3\%$, discounting curve is flat $2\%$ (both continuously compounded).

# Mathematical background
Fast forward, to discount a CMS coupon that is based on swap rate $S$ that is set at $T_0$ (the underlying swap terminates at $T_N$) and is paid at $T_p\ge T_0$ ('CMS coupon'), we need to compute expectation of the swap rate under $T_p$-forward measure. That gives:
\begin{equation}
{R_{\text{CMS}}}(0) = {\mathbb{E}^{{\mathbb{Q}_{{T_p}}}}}\left[ {S({T_0})} \right] = \frac{{A(0)}}{{B(0,{T_p})}}{\mathbb{E}^{{\mathbb{Q}_A}}}\left[ {\frac{{B({T_0},{T_p})}}{{A({T_0})}}S({T_0})} \right],
\end{equation}
where
\begin{equation}
A(t) = \sum\limits_{i = 1}^N {{\delta _i}B(t,{T_i})},
\end{equation}
is the annuity of the 'underlying' swap.
The computational complexity of CMS comes from the ${\frac{{B({T_0},{T_p})}}{{A({T_0})}}}$ term in under the annuity-measure expectation. To deal with this term, typically some kind of approximation is used. Commonly, the approximation expresses this term as some affine function of $S$ (which is easy to deal with since $S$ is martingale under the annuity measure). This no different for the TSR approach where the term is approximated the following expression:
\begin{equation}
\frac{{B({T_0},{T_p})}}{{A({T_0})}} = a({T_p})S({T_0}) + b({T_p}),
\end{equation}
where
\begin{eqnarray*}
a({T_p}) &=& \frac{{B(0,{T_p})(\gamma  - G({T_p} - {T_0}))}}{{B(0,{T_N})G({T_N} - {T_0}) + A(0)S(0)\gamma }}, \\
G(\tau ) &=& \frac{{1 - {e^{ - \varkappa \tau }}}}{\varkappa }\\
\gamma  &=& \frac{{\sum\limits_{i = 1}^N {{\delta _i}B(0,{T_i})G({T_i} - {T_0})} }}{{A(0)}}.
\end{eqnarray*}
$\varkappa$ is a mean-reversion parameter that needs to be supplied.

The $t=0$ NPV of the CMS coupon in LinearTSR model is then 
\begin{eqnarray*}
V(0) &=& B(0,{T_p})(T_p - T_0){\mathbb{E}^{{\mathbb{Q}_{{T_p}}}}}\left[ {S({T_0})} \right]\\
     &=& B(0,{T_p})(T_p - T_0){R_{\text{CMS}}}(0)\\
     &=& B(0,{T_p})(T_p - T_0)\left( S(0) + \frac{{A(0)}}{{B(0,{T_p})}}a({T_p}){\mathbb{V}}^{\mathbb{Q}_A} [S({T_0})] \right).
\end{eqnarray*}
where ${\mathbb{V}}^{\mathbb{Q}_A} [S({T_0})]$ is variance of $S(T_0)$ under the annuity measure $\mathbb{Q}_A$ (nuder which $S$ is a martingale).  

Under normal model
\begin{equation}
{\mathbb{V}}^{\mathbb{Q}_A} [S({T_0})] = {\sigma ^2}{T_0}.
\end{equation}
Under ($s$-shifted) lognormal model
\begin{equation*}
{\mathbb{V}}^{\mathbb{Q}_A} [S({T_0})] = {(S(0) + s)^2}({e^{{\sigma ^2}{T_0}}} - 1).
\end{equation*}

References:
- Borst, C. 2014. _The efficient pricing of CMS and CMS spread derivatives_.  https://repository.tudelft.nl/islandora/object/uuid:71ac754b-23ad-4224-ad50-6828527d6d0d/datastream/OBJ/download,
- Zeng, Y. 2015. _Convexity Adjustment: A User’s Guide_. https://www.quantsummaries.com/Zeng_convexity_adj.pdf
- Piterbarg, V. 2010. _Interest Rate Modeling. Volume 3: Products and Risk Management_ (sectios 16.6., 16.3. and sections 16.3.2 and 16.6.4 in particular)

# Definitions

In [2]:
today = ql.Date(20, 10, 2022)
ql.Settings.instance().evaluationDate = today

# interest-rate curves
discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.TARGET(), ql.QuoteHandle(ql.SimpleQuote(0.02)), ql.Actual360()))
forecast_curve = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.TARGET(), ql.QuoteHandle(ql.SimpleQuote(0.03)), ql.Actual360()))

In [3]:
sigma = 0.60 # volatility of the forward swap rate in the shifted lognormal model
shift = 0.02 # shift in the shifted lognormal model
kappa = 0.05 # mean-reversion speed

T0_period = ql.Period('2Y')
Tp_period = ql.Period('3Y')

T0 = today + T0_period # start date of CMS (and start day of the accrual period)
Tp = today + Tp_period # end  date of CMS accrual period; pay date of CMS coupon

fixingDays = 0

swap_tenor = ql.Period('5Y') # this is TN-T0: tenor of the underlying swap on which CMS rate is paid
TN =  T0 + swap_tenor # maturity if the underlying swap

print(f'Today is t = {today.to_date()}. We are pricing a CMS coupon with the following details:')
print(f'The underlying {swap_tenor} swap starts on T0 = {T0.to_date()} and ends by TN = {TN.to_date()}')
print(f'CMS accrual period is [T0 = {T0.to_date()}, Tp = {Tp.to_date()}]. The CMS coupon is paid out by Tp = {Tp.to_date()}')

Today is t = 2022-10-20. We are pricing a CMS coupon with the following details:
The underlying 5Y swap starts on T0 = 2024-10-20 and ends by TN = 2029-10-20
CMS accrual period is [T0 = 2024-10-20, Tp = 2025-10-20]. The CMS coupon is paid out by Tp = 2025-10-20


# CMS coupon valuation using `ql.LinearTsrPricer`

In [4]:
# use the ready-made index for EUR swaps, or build a custom IRS index below
swapIndex  = ql.EuriborSwapIsdaFixA(swap_tenor,forecast_curve, discount_curve)
#swapIndex = ql.SwapIndex('EUR', ql.Period(swap_tenor), fixingDays, ql.EURCurrency(), ql.TARGET(), ql.Period('1Y'), ql.ModifiedFollowing, ql.Actual360(), ql.Euribor6M(forecast_curve), discount_curve)

nominal = 1.0
cms            = ql.CmsCoupon(Tp, nominal, T0, Tp, fixingDays, swapIndex)
swaptionVol    = ql.ConstantSwaptionVolatility(fixingDays, ql.TARGET(), ql.ModifiedFollowing, ql.QuoteHandle(ql.SimpleQuote(sigma)), ql.Actual365Fixed(), ql.ShiftedLognormal, shift)
swvol_handle   = ql.SwaptionVolatilityStructureHandle(swaptionVol)
mean_reversion = ql.QuoteHandle(ql.SimpleQuote(kappa))
cms_pricer     = ql.LinearTsrPricer(swvol_handle, mean_reversion)
cms.setPricer(cms_pricer)


print(f'CMS rate R(0)={cms.rate()}, which includes {cms.convexityAdjustment()} convexity adjustment')
print(f'The NPV of the CMS coupon is {cms.price(discount_curve)} (via cms.price), or equivalently {ql.CashFlows.npv(ql.Leg([cms]), discount_curve, True)} via NPV of leg with a single cash-flow')

CMS rate R(0)=0.0358722347936681, which includes 0.005050186635780569 convexity adjustment
The NPV of the CMS coupon is 0.033753182301828354 (via cms.price), or equivalently 0.033753182301828354 via NPV of leg with a single cash-flow


# CMS coupon valuation using the formulas above

In [5]:
# define the underlying swap object, coupon rate doesn't matter as it won't be used
swp = ql.MakeVanillaSwap(swap_tenor, ql.Euribor6M(forecast_curve), 0.03, T0_period, 
                         settlementDays = 0,
                         fixedLegCalendar = ql.TARGET(), 
                         fixedLegTenor = ql.Period('1Y'), 
                         discountingTermStructure = discount_curve,
                         fixedLegDayCount = ql.Thirty360(1), 
                         floatingLegDayCount = ql.Actual360())
S = swp.fairRate() # forward-swap rate, as seen from t=0

In [6]:
G = lambda tau, kappa: (1-np.exp(-kappa*tau))/kappa if kappa>0 else tau
yearfrac = lambda date1, date2: ql.Thirty360(0).yearFraction(date1,date2)

annuity = sum([yearfrac(swp.fixedSchedule()[idx-1],date)*discount_curve.discount(date) for idx, date in enumerate(swp.fixedSchedule()) if idx >0])
gamma   = sum([yearfrac(swp.fixedSchedule()[idx-1],date)*discount_curve.discount(date)*G(yearfrac(T0,date), kappa) 
               for idx, date in enumerate(swp.fixedSchedule()) if idx >0])/annuity

a = discount_curve.discount(Tp)*(gamma - G(yearfrac(T0,Tp), kappa))/(discount_curve.discount(TN)*G(yearfrac(T0,TN), kappa) + annuity*S*gamma)
var      = (S+shift)**2 * (np.exp(sigma**2* yearfrac(today,T0)) - 1) # var = (sigma**2 * yearfrac(today,T0)) <- this would be valid for a normal model
cms_rate = S + annuity/discount_curve.discount(Tp) * a * var

In [7]:
print(f'CMS rate R(0)={cms_rate}, which includes {cms_rate - S} convexity adjustment')

CMS rate R(0)=0.03589295079314921, which includes 0.005070929951134999 convexity adjustment
